In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops

In [ ]:
preprocessed_glcm_path = r"D:\MEDLF\DATA\Preprocessed_for_GLCM"
glcm_features_csv = r"D:\MEDLF\DATA\GLCM_Features.csv"

In [ ]:
DISTANCES = [1, 2, 3]  # Distances for GLCM computation
ANGLES = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]  # Angles for GLCM computation
GLCM_PROPS = ["contrast", "dissimilarity", "homogeneity", "energy", "correlation", "ASM"]


In [ ]:
def compute_glcm_features(image):
    features = {}
    # Compute GLCM
    glcm = graycomatrix(
        image, distances=DISTANCES, angles=ANGLES, levels=256, symmetric=True, normed=True
    )
    # Extract properties
    for prop in GLCM_PROPS:
        prop_values = graycoprops(glcm, prop)
        # Average values across all distances and angles
        features[prop] = np.mean(prop_values)
    return features


In [ ]:
def process_class_images(class_path, class_name, source_label):
    data = []
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            features = compute_glcm_features(image)
            features["class"] = class_name
            features["image_name"] = image_file
            features["source"] = source_label  # Indicate "Augmented" or "Original"
            data.append(features)
        else:
            print(f"Error loading image: {image_path}")
    return data


In [ ]:
def extract_glcm_features(dataset_path):
    all_data = []
    for folder_name in ["Augmented", "Original"]:
        folder_path = os.path.join(dataset_path, folder_name)
        if not os.path.exists(folder_path):
            print(f"Folder not found: {folder_path}")
            continue
        print(f"Processing folder: {folder_name}")
        for class_folder in os.listdir(folder_path):
            class_path = os.path.join(folder_path, class_folder)
            if os.path.isdir(class_path):
                print(f"Processing class: {class_folder}")
                class_data = process_class_images(class_path, class_folder, folder_name)
                all_data.extend(class_data)
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    return df


In [ ]:
if __name__ == "__main__":
    print("Extracting GLCM features from preprocessed data...")
    glcm_features_df = extract_glcm_features(preprocessed_glcm_path)
    print(f"Saving GLCM features to {glcm_features_csv}...")
    glcm_features_df.to_csv(glcm_features_csv, index=False)
    print("GLCM feature extraction complete!")